# Interesting observations

In [ ]:
from types import SimpleNamespace
import numpy as np
import holoviews as hv
import pandas as pd
hv.notebook_extension()
%opts Scatter (color='b') Path [show_legend=True aspect='square']
pd.set_option('display.max_columns', None)

def dims(df):
    return SimpleNamespace(phi=hv.Dimension(r'$\phi$', unit='rad'),
                           phi_c=hv.Dimension(r'$\phi_c$', unit='rad'),
                           I=hv.Dimension('$I$', unit=r'meV / $\Phi_0$'),
                           I_c=hv.Dimension('$I_c$', unit=r'meV / $\Phi_0$'),
                           B=hv.Dimension('$B$', unit='T', values=df['B_x']),
                           T=hv.Dimension('$T$', unit='K', values=sorted(df['T'])), 
                           L=hv.Dimension('$L$', unit='nm', values=sorted(df['L'])), 
                           g=hv.Dimension('$g$-factor', values=df['g']), 
                           alpha=hv.Dimension(r'$\alpha$', unit='mev nm', values=df['alpha']), 
                           mu=hv.Dimension(r'$\mu$', unit='meV', values=df['mu']), 
                           a=hv.Dimension('$a$', unit='nm', values=df['a']),
                           orbital=hv.Dimension('orbital', values=[True, False]),
                           leads=hv.Dimension('leads', values=[True, False]),
                           zoom=hv.Dimension('zoom', values=[1, 5, 10, 25, 50, 75, 100, 200]))

def dynamic_map(fname, label=None):
    df = pd.read_hdf(fname)
    d = dims(df)
    gb = df.groupby(['T', 'L', 'with_leads', 'orbital', 'g', 'alpha', 'mu', 'a'])
    groups = dict(list(gb))

    if label is None:
        label = fname

    def current_curve(T, L, leads, orbital, g, alpha, mu, a, zoom):
        gr = groups[(T, L, leads, orbital, g, alpha, mu, a)]
        return hv.Path((gr['B_x'], gr['current_c']), kdims=[d.B, d.I_c], label=label)[:, 0:zoom]

    def phase_curve(T, L, leads, orbital, g, alpha, mu, a, zoom):
        gr = groups[(T, L, leads, orbital, g, alpha, mu, a)]
        return hv.Path((gr['B_x'], gr['phase_c']), kdims=[d.B, d.phi_c], label=label)[:, -np.pi:np.pi]

    def current_phase(B_x, T, L, leads, orbital, g, alpha, mu, a, zoom):
        gr = groups[(T, L, leads, orbital, g, alpha, mu, a)]
        gr = gr[gr['B_x'] == B_x]
        phases = np.array(gr['phases'])[0]
        currents = np.array(gr['currents'])[0]
        return (hv.Path((phases, currents), kdims=[d.phi, d.I], label=label)[:, -zoom:zoom] *
                hv.Scatter((gr['phase_c'], gr['current_c'])))

    kdims = [d.T, d.L, d.leads, d.orbital, d.g, d.alpha, d.mu, d.a, d.zoom]

    return (hv.DynamicMap(current_curve, kdims=kdims),
            hv.DynamicMap(phase_curve, kdims=kdims),
            hv.DynamicMap(current_phase, kdims=[d.B] + kdims))

In [ ]:
dm = dynamic_map('data/all_data_circular_A_in_SM.hdf')

In [ ]:
dm[0] + dm[1]

In [ ]:
dm[2]

# Without Zeeman and with orbital effects, spin-orbit has little effect on $I_c$

# With spin-orbit and with orbital effects, Zeeman has little effect on $I_c$

# Without orbital effects and without spin-orbit, the $B$ dependence scales both with $L$ and $\mu$

# In single mode regime, Zeeman has the predominant effect on $I_c$

# Both in the single mode and multimode regime, the orbital effect has the predominant effect on $I_c$

# $0-\pi$ transitions?
* $L=640, \mu=50$
* $L=640, \mu=40$
* $L=160, \mu=30$
